In [1]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-xsil005i
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-xsil005i
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from tqdm import tqdm
from glob import glob
from PIL import Image
import pandas as pd
import os
from os.path import join, basename
import numpy as np
import torch
from torchvision import transforms
from typing import List, Tuple, Any
import gc

dataset_path = "/kaggle/input/lost-in-the-museum-v2/archive/kaggle_dataset/kaggle_dataset"

all_images = sorted(glob(join(dataset_path, "*.png")))
print(f"All images: {len(all_images)}")

All images: 20000


In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

Using device: cuda


In [4]:
import clip

# OpenCLIP (ViT-L/14)
# We use the OpenCLIP library directly for this.
CLIP_MODEL_NAME = "ViT-L/14"
CLIP_PRETRAINED_DATASET = "laion2b_s34b_b88k"
clip_model, clip_preprocess = clip.load(
    CLIP_MODEL_NAME, 
    device="cpu", # Will be moved to the target device later
    jit=False, 
    download_root='/kaggle/temp/clip_cache/' # Use temp directory for Kaggle
)
clip_model.to(DEVICE).eval()

100%|████████████████████████████████████████| 890M/890M [00:03<00:00, 250MiB/s]


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)


In [5]:
# DinoV2 (ViT-L/14)
dinov2_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14_reg')
dinov2_model.to(DEVICE).eval()

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitl14/dinov2_vitl14_reg4_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitl14_reg4_pretrain.pth
100%|██████████| 1.13G/1.13G [00:12<00:00, 98.3MB/s]


DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-23): 24 x NestedTensorBlock(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=True)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )
  )
  (norm): LayerNorm((1024,), eps=1e-06, element

In [6]:
dataset_path = "/kaggle/input/lost-in-the-museum-v2/archive/kaggle_dataset/kaggle_dataset"
all_images = sorted(glob(join(dataset_path, "*.png")))
print(f"All images found: {len(all_images)}")

def load_image_raw(image_path: str):
    """Load a single image without initial transformation."""
    try:
        return Image.open(image_path).convert('RGB')
    except Exception:
        return None

# DINOv2 Preprocessor (Standard ImageNet normalization and sizing)
# ViT-B/14 typically uses 224x224 or 518x518. We use 224 for balance.
DINOV2_RESIZE = 224 
dinov2_preprocess = transforms.Compose([
    transforms.Resize(DINOV2_RESIZE, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(DINOV2_RESIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

# --- 3. CHUNK-BASED FEATURE EXTRACTION FUNCTION ---

def extract_features_chunk(
    image_chunk: List[str], 
    dinov2_model: torch.nn.Module, 
    clip_model: torch.nn.Module, 
    clip_preprocess: Any, 
    device: str
) -> Tuple[List[str], List[np.ndarray]]:
    """Extracts features for a list of images using DINOv2 and OpenCLIP."""
    
    features_list = []
    valid_image_paths = []
    
    with torch.no_grad():
        for i, image_path in enumerate(image_chunk):
            
            raw_image = load_image_raw(image_path)
            if raw_image is None:
                continue
            
            # --- DINOv2 Feature Extraction ---
            try:
                dinov2_input = dinov2_preprocess(raw_image).unsqueeze(0).to(device)
                
                # Get patch tokens (including CLS and registers)
                dinov2_output = dinov2_model.forward_features(dinov2_input)
                # Take all patch tokens for GAP, typically in 'x_norm_patchtokens'
                all_tokens = dinov2_output['x_norm_patchtokens'].squeeze(0) 
                
                # Global Average Pooling (GAP) across all tokens for global feature
                dinov2_feature = all_tokens.mean(dim=0)
                dinov2_feature = dinov2_feature.cpu().numpy()
                
                # Cleanup PyTorch tensors
                del dinov2_input, dinov2_output, all_tokens
                
            except Exception as e:
                print(f"DINOv2 error for {basename(image_path)}: {e}")
                continue

            # --- OpenCLIP Feature Extraction ---
            try:
                clip_input = clip_preprocess(raw_image).unsqueeze(0).to(device)
                clip_feature = clip_model.encode_image(clip_input)
                clip_feature = clip_feature.cpu().numpy().squeeze(0)
                
                # Cleanup PyTorch tensors
                del clip_input
                
            except Exception as e:
                print(f"CLIP error for {basename(image_path)}: {e}")
                continue

            # --- Normalization and Fusion ---
            dinov2_feature_norm = dinov2_feature / np.linalg.norm(dinov2_feature)
            clip_feature_norm = clip_feature / np.linalg.norm(clip_feature)
            combined_features = np.concatenate([dinov2_feature_norm, clip_feature_norm])

            # --- Store and Cleanup ---
            features_list.append(combined_features)
            valid_image_paths.append(basename(image_path))
            
            # Aggressive cleanup of raw data and intermediate numpy arrays
            del dinov2_feature, clip_feature, dinov2_feature_norm, clip_feature_norm, combined_features
            del raw_image
            
            # Periodically empty CUDA cache to clear fragmented VRAM
            if (i + 1) % 50 == 0 and torch.cuda.is_available():
                torch.cuda.empty_cache()

    return valid_image_paths, features_list

# --- 4. MAIN CHUNKING AND SAVING LOGIC ---

def run_feature_extraction_and_save(all_images: List[str], output_path: str, chunk_size: int = 500):
    """Iterates over images in chunks, extracts features, and appends to disk."""

    if os.path.exists(output_path):
        os.remove(output_path)
        print(f"Existing file {output_path} removed.")
        
    print(f"\nStarting feature extraction in chunks of {chunk_size}...")
    
    total_images = len(all_images)
    
    for i, start_index in enumerate(tqdm(range(0, total_images, chunk_size), desc="Processing Chunks")):
        end_index = min(start_index + chunk_size, total_images)
        image_chunk = all_images[start_index:end_index]
        
        # --- Extract features for the current chunk ---
        valid_paths, features_list = extract_features_chunk(
            image_chunk, 
            dinov2_model, clip_model, clip_preprocess, DEVICE
        )
        
        if not features_list:
            print(f"Warning: Chunk {i+1} yielded no valid features. Skipping save.")
            continue
            
        # --- Create DataFrame and Save ---
        N = features_list[0].shape[0]
        column_names = [f"feature_{i}" for i in range(N)]
        df = pd.DataFrame(features_list, columns=column_names)
        df.insert(0, 'image_name', valid_paths)
        df['ID'] = df['image_name']
        
        header = (i == 0) # Write header only for the first chunk
        mode = 'w' if (i == 0) else 'a' # 'w' for first chunk, 'a' for append
        
        # Save to CSV, appending subsequent chunks
        df.to_csv(output_path, mode=mode, header=header, index=False)
        
        # --- FINAL MEMORY CLEANUP FOR THE CHUNK ---
        del features_list, valid_paths, df
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

# --- 5. EXECUTION ---
output_csv_path = "/kaggle/working/submission.csv"
CHUNK_SIZE = 100 # Keep this low (e.g., 100-500) until stable

run_feature_extraction_and_save(all_images, output_csv_path, CHUNK_SIZE)

print(f"\n✨ Feature extraction complete. Final output saved to {output_csv_path}")

All images found: 20000

Starting feature extraction in chunks of 100...


Processing Chunks: 100%|██████████| 200/200 [42:32<00:00, 12.76s/it]


✨ Feature extraction complete. Final output saved to /kaggle/working/submission.csv
